<a href="https://colab.research.google.com/github/YianKim/2022_uncertainty_aware_semisupervise/blob/main/Keras_UncertaintyBootstrap_SVHN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 9.5 MB/s 


In [2]:
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import clone_model

import PIL
from PIL import Image

import pickle
import random
from tqdm import tqdm
from collections import Counter

from keras.layers.core import Lambda
from keras import backend as K

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import BatchNormalization
from keras.regularizers import l2
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import MaxPooling2D, AveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Reshape
from keras import optimizers
from keras.callbacks import *
from sklearn.metrics import *
from keras.models import load_model
import tensorflow_addons as tfa

from torchvision import transforms

import tensorflow as tf
import tensorflow.keras.backend as backend
import math
import gc

# SVHN

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from scipy.io import loadmat
train_raw = loadmat('/content/drive/MyDrive/SVHN/train_32x32.mat')
test_raw = loadmat('/content/drive/MyDrive/SVHN/test_32x32.mat')

In [5]:
def dummy_labels(labels):
  zero_labels = np.zeros([labels.shape[0], 10], np.int8)  
  for i in range(labels.shape[0]):
    zero_labels[i][labels[i]] = 1
  return(zero_labels)

In [6]:
train_images = train_raw['X']
train_labels = train_raw['y']

test_images = test_raw['X']
test_labels = dummy_labels(test_raw['y']-1)

train_images = train_images.swapaxes(2,3).swapaxes(1,2).swapaxes(0,1)
test_images = test_images.swapaxes(2,3).swapaxes(1,2).swapaxes(0,1)

In [7]:
temp = [0,0,0,0,0,0,0,0,0,0]
label_indx = []
unlabel_indx = []

for i in range(73257) :
  if temp[(train_labels).reshape([-1])[i]-1] < 25 :
    temp[(train_labels).reshape([-1])[i]-1] += 1
    label_indx.append(i)
  else :
    unlabel_indx.append(i)

In [8]:
lbl_train_images = train_images[label_indx]
lbl_train_labels = dummy_labels(train_labels[label_indx]-1)

In [9]:
ubl_train_images = train_images[unlabel_indx]
ubl_train_labels = dummy_labels(train_labels[unlabel_indx]-1)

#MAin


In [10]:
class SGDR(Callback):

    def __init__(self, min_lr=0.0, max_lr=0.03, base_epochs=20, mul_epochs=2):
        super(SGDR, self).__init__()

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.base_epochs = base_epochs
        self.mul_epochs = mul_epochs

        self.cycles = 0.
        self.cycle_iterations = 0.
        self.trn_iterations = 0.

        self._reset()

    def _reset(self, new_min_lr=None, new_max_lr=None,
               new_base_epochs=None, new_mul_epochs=None):
        """Resets cycle iterations."""
        
        if new_min_lr != None:
            self.min_lr = new_min_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_base_epochs != None:
            self.base_epochs = new_base_epochs
        if new_mul_epochs != None:
            self.mul_epochs = new_mul_epochs
        self.cycles = 0.
        self.cycle_iterations = 0.
        
    def sgdr(self):
        
        cycle_epochs = self.base_epochs * (self.mul_epochs ** self.cycles)
        tide = ((self.cycles == 0) * 1) * (self.cycle_iterations*self.max_lr + (self.base_epochs - self.cycle_iterations)*self.min_lr) / self.base_epochs + ((self.cycles != 0) * 1)*(self.min_lr + 0.5 * (self.max_lr - self.min_lr) * (1 + np.cos(np.pi * (self.cycle_iterations + 1) / cycle_epochs)))
        return tide
        
    def on_train_begin(self, logs=None):
        
        if self.cycle_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.max_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.sgdr())
            
    def on_epoch_end(self, epoch, logs=None):
        
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)
        
        self.trn_iterations += 1
        self.cycle_iterations += 1
        if self.cycle_iterations >= self.base_epochs * (self.mul_epochs ** self.cycles):
            self.cycles += 1
            self.cycle_iterations = 0
            K.set_value(self.model.optimizer.lr, self.max_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.sgdr())

In [11]:
def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))

def create_cnn_1():
  conv1a = Conv2D(32, (3,3), padding = 'same')
  bn1a = BatchNormalization()
  conv1b = Conv2D(32, (3,3), padding = 'same')
  bn1b = BatchNormalization()
  conv1c = Conv2D(32, (3,3), padding = 'same')
  bn1c = BatchNormalization()
  pl1 = MaxPooling2D(2, 2)
  MCdrop1 = PermaDropout(0.4)

  conv2a = Conv2D(64, (3,3), padding = 'same')
  bn2a = BatchNormalization()
  conv2b = Conv2D(64, (3,3), padding = 'same')
  bn2b = BatchNormalization()
  conv2c = Conv2D(64, (3,3), padding = 'same')
  bn2c = BatchNormalization()
  pl2 = MaxPooling2D(2, 2)
  MCdrop2 = PermaDropout(0.4)

  conv3a = Conv2D(128, (3,3))
  bn3a = BatchNormalization()
  conv3b = Conv2D(128, (3,3))
  bn3b = BatchNormalization()
  conv3c = Conv2D(128, (3,3))
  bn3c = BatchNormalization()
  pl3 = AveragePooling2D(4, 1)
  MCdrop3 = PermaDropout(0.4)

  fc1 = Dense(128)
  fc2 = Dense(10)
  activ = keras.layers.LeakyReLU(0.1)

  model = Sequential([
                      keras.Input(shape=(32, 32, 3)), 
                      tfa.layers.WeightNormalization(conv1a), bn1a, activ,
                      tfa.layers.WeightNormalization(conv1b), bn1b, activ,
                      # tfa.layers.WeightNormalization(conv1c), bn1c, activ,
                      pl1, MCdrop1,

                      tfa.layers.WeightNormalization(conv2a), bn2a, activ,
                      tfa.layers.WeightNormalization(conv2b), bn2b, activ,
                      # tfa.layers.WeightNormalization(conv2c), bn2c, activ,
                      pl2, MCdrop2,

                      tfa.layers.WeightNormalization(conv3a), bn3a, activ,
                      tfa.layers.WeightNormalization(conv3b), bn3b, activ,
                      # tfa.layers.WeightNormalization(conv3c), bn3c, activ,
                      pl3, MCdrop3, Flatten(),
                      
                      fc1, activ, fc2
                      ])
  
  return model

def create_cnn_2():
  conv1a = Conv2D(32, (3,3), padding = 'same')
  bn1a = BatchNormalization()
  conv1b = Conv2D(32, (3,3), padding = 'same')
  bn1b = BatchNormalization()
  conv1c = Conv2D(32, (3,3), padding = 'same')
  bn1c = BatchNormalization()
  pl1 = MaxPooling2D(2, 2)
  MCdrop1 = PermaDropout(0.4)

  conv2a = Conv2D(64, (3,3), padding = 'same')
  bn2a = BatchNormalization()
  conv2b = Conv2D(64, (3,3), padding = 'same')
  bn2b = BatchNormalization()
  conv2c = Conv2D(64, (3,3), padding = 'same')
  bn2c = BatchNormalization()
  pl2 = MaxPooling2D(2, 2)
  MCdrop2 = PermaDropout(0.4)

  conv3a = Conv2D(128, (3,3))
  bn3a = BatchNormalization()
  conv3b = Conv2D(128, (3,3))
  bn3b = BatchNormalization()
  conv3c = Conv2D(128, (3,3))
  bn3c = BatchNormalization()
  pl3 = AveragePooling2D()
  MCdrop3 = PermaDropout(0.4)

  fc1 = Dense(128)
  fc2 = Dense(10)
  activ = keras.layers.LeakyReLU(0.1)

  model = Sequential([
                      keras.Input(shape=(32, 32, 3)), 
                      tfa.layers.WeightNormalization(conv1a), bn1a, activ,
                      tfa.layers.WeightNormalization(conv1b), bn1b, activ,
                      tfa.layers.WeightNormalization(conv1c), bn1c, activ,
                      pl1, MCdrop1,

                      tfa.layers.WeightNormalization(conv2a), bn2a, activ,
                      tfa.layers.WeightNormalization(conv2b), bn2b, activ,
                      tfa.layers.WeightNormalization(conv2c), bn2c, activ,
                      pl2, MCdrop2,

                      tfa.layers.WeightNormalization(conv3a), bn3a, activ,
                      tfa.layers.WeightNormalization(conv3b), bn3b, activ,
                      tfa.layers.WeightNormalization(conv3c), bn3c, activ,
                      pl3, MCdrop3, Flatten(),
                      
                      fc1, activ, fc2
                      ])
  
  return model

In [12]:
def compile_cnn(model):

  if control_lrschedule == 0 :
    opt = keras.optimizers.Adam(0.003)
  else :
    opt = keras.optimizers.Adam(0.001)

  model.compile(
    optimizer = opt,
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
  )

  return model

def cnn_13():

  if control_lrschedule == 0:
    model = create_cnn_1()
  if control_lrschedule == 1:
    model = create_cnn_2()
  model = compile_cnn(model)

  return model

def fit_and_labeling_cnn_13(Epoch, Batch):

  lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
  early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=30, mode='auto')
  sgdr = SGDR(min_lr=0.0, max_lr=0.03, base_epochs=20) #스케줄러

  model.fit(
      x=X,
      y=y,
      epochs=Epoch,
      verbose=1,
      batch_size=Batch
      )
    
  model_test_eval(model, test_images, test_labels)
  T = 1

  for predsamples in (range(10)):
    if predsamples == 0 :
      predictions = np.array(tf.nn.softmax(model.predict(ubl_train_images)/T))
      predictions = predictions.reshape((1,) + predictions.shape)
    else:
      pred = np.array(tf.nn.softmax(model.predict(ubl_train_images)/T))
      pred = pred.reshape((1,) + pred.shape)
      predictions = np.concatenate((predictions, pred))

  return predictions

def model_test_eval(model, test_images, test_labels):
  T = 1
  pred = np.array(tf.nn.softmax(model.predict(test_images)/T))
  for i in range(1,10):
    pred += np.array(tf.nn.softmax(model.predict(test_images)))
  acc = (np.argmax(pred,axis=1) == np.argmax(test_labels,axis=1))*1
  acc = sum(acc)/len(acc)
  print("test set 성능 : " + str(acc))

In [13]:
def basic_augmentation(imagearray):
  image = Image.fromarray(imagearray)
  tr2 = transforms.RandomRotation(10)
  tr3 = transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5)
  image = tr2(tr3(image))
  return(np.array(image))

def makeaugs(n, input):
  augs = []
  for j in range(n):
    for i in input:
      augs.append(basic_augmentation(np.array(i, np.uint8)))
  return(np.array(augs))

In [15]:
first = time.time()
control_lrschedule = 0
model = cnn_13()
X = lbl_train_images
y = lbl_train_labels

X = np.concatenate([makeaugs(20, X), X])
y = np.concatenate([y,y,y,y,y,
                    y,y,y,y,y,
                    y,y,y,y,y,
                    y,y,y,y,y, y])

print("< preparation evaluation >")
predictions = fit_and_labeling_cnn_13(100, 64)
print(time.time() - first)

< preparation evaluation >
Epoch 1/100
83/83 [==============================] - 3s 12ms/step - loss: 2.3177 - accuracy: 0.1326
Epoch 2/100
83/83 [==============================] - 1s 11ms/step - loss: 2.0310 - accuracy: 0.2808
Epoch 3/100
83/83 [==============================] - 1s 11ms/step - loss: 1.3997 - accuracy: 0.5284
Epoch 4/100
83/83 [==============================] - 1s 13ms/step - loss: 0.8571 - accuracy: 0.7190
Epoch 5/100
83/83 [==============================] - 1s 15ms/step - loss: 0.5595 - accuracy: 0.8223
Epoch 6/100
83/83 [==============================] - 1s 15ms/step - loss: 0.4573 - accuracy: 0.8531
Epoch 7/100
83/83 [==============================] - 1s 10ms/step - loss: 0.3394 - accuracy: 0.8899
Epoch 8/100
83/83 [==============================] - 1s 10ms/step - loss: 0.3325 - accuracy: 0.8928
Epoch 9/100
83/83 [==============================] - 1s 10ms/step - loss: 0.1963 - accuracy: 0.9417
Epoch 10/100
83/83 [==============================] - 1s 11ms/step - loss

In [ ]:
schedule = list(2**np.array(list(range(5)))/(2**5))
schedule = [[i,i,i,i,i] for i in schedule]
schedule = np.array(schedule).reshape([-1])

alpha = 0.6
control_lrschedule = 1

model = cnn_13()
epoch = 25

for iter in range(len(schedule)):
    pseudo = np.argmax(np.mean(predictions, axis=0), axis=1)
    conf = np.max(np.mean(predictions, axis=0), axis=1)
    uncert = np.std(predictions, axis=0)
    uncert = np.array([uncert[i][pseudo[i]] for i in range(len(pseudo))])
    cert = 1-uncert
    
    ubl_pseudo_labels = []
    for i in pseudo:
        temp = [0,0,0,0,0,0,0,0,0,0]
        temp[i] = 1
        ubl_pseudo_labels.append(temp)
    ubl_pseudo_labels = np.array(ubl_pseudo_labels)

    score = alpha*conf + (1-alpha)*cert
    score = (score-min(score))/(max(score)-min(score))+0.0001
    score = np.exp(score/schedule[iter])
    score = score/sum(score)

    indx = np.random.choice(len(score), 300000, p = score)

    X = ubl_train_images[indx]
    y = ubl_pseudo_labels[indx]

    X = np.concatenate([lbl_train_images, X])
    y = np.concatenate([lbl_train_labels, y])

    augimage = makeaugs(1, X)
    X = augimage

    print("< iter "+str(iter)+" evaluation >")
    predictions = fit_and_labeling_cnn_13(epoch, 64)
    print(time.time() - first)
    
    epoch = 10
    del augimage, X, y
    gc.collect()

< iter 0 evaluation >
Epoch 1/25
4692/4692 [==============================] - 75s 15ms/step - loss: 0.1199 - accuracy: 0.9616
Epoch 2/25
4692/4692 [==============================] - 71s 15ms/step - loss: 0.0355 - accuracy: 0.9887
Epoch 3/25
4692/4692 [==============================] - 71s 15ms/step - loss: 0.0273 - accuracy: 0.9912
Epoch 4/25
4692/4692 [==============================] - 71s 15ms/step - loss: 0.0219 - accuracy: 0.9930
Epoch 5/25
4692/4692 [==============================] - 71s 15ms/step - loss: 0.0181 - accuracy: 0.9941
Epoch 6/25
4692/4692 [==============================] - 70s 15ms/step - loss: 0.0168 - accuracy: 0.9946
Epoch 7/25
4692/4692 [==============================] - 70s 15ms/step - loss: 0.0145 - accuracy: 0.9953
Epoch 8/25
4692/4692 [==============================] - 71s 15ms/step - loss: 0.0126 - accuracy: 0.9959
Epoch 9/25
4692/4692 [==============================] - 70s 15ms/step - loss: 0.0120 - accuracy: 0.9961
Epoch 10/25
4692/4692 [===================